In [4]:
!conda install -y fastcore -c fastai

Retrieving notices: ...working... done
Solving environment: done

## Package Plan ##

  environment location: /home/aadi/miniconda3

  added / updated specs:
    - fastcore


The following NEW packages will be INSTALLED:

  fastcore           fastai/noarch::fastcore-1.5.27-py_0 




Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [1]:
import sys
import os

os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-1.8.0-openjdk-amd64'
os.environ['PYSPARK_PYTHON'] = '/home/aadi/miniconda3/envs/pyspark_env/bin/python' 
os.environ['PYSPARK_DRIVER_PYTHON'] = '/home/aadi/miniconda3/envs/pyspark_env/bin/python' 


In [2]:
import pyspark

spark = pyspark.sql.SparkSession.Builder().getOrCreate()

23/03/01 14:03:31 WARN Utils: Your hostname, debian resolves to a loopback address: 127.0.1.1; using 192.168.100.254 instead (on interface wlp5s0)
23/03/01 14:03:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/03/01 14:03:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import numpy as np
import pandas as pd
from pandas import DataFrame


N = 1000
df = DataFrame({
    'Gender': np.random.choice(['M', 'F'], size=N),
    'Age': np.random.randint(low=10, high=100, size=N),
    'Score': np.random.uniform(0, 1, size=N),
    'Binary_Target': np.random.binomial(n=1, p=0.2, size=N)
})

df = df.assign(Pred=np.where(df.Score > 0.5, 1, 0))
data = spark.createDataFrame(df)
data.show()

/home/aadi/miniconda3/envs/spark_env/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+------+---+-------------------+-------------+----+
|Gender|Age|              Score|Binary_Target|Pred|
+------+---+-------------------+-------------+----+
|     F| 13| 0.8020636228021221|            0|   1|
|     F| 98|0.17461547993526194|            1|   0|
|     M| 20| 0.7345702319301879|            1|   1|
|     M| 29| 0.9666501783296002|            0|   1|
|     F| 37| 0.9798025598241514|            0|   1|
|     M| 88| 0.1920040322549238|            0|   0|
|     F| 33| 0.6642727956768272|            1|   1|
|     F| 88| 0.8674975138997422|            1|   1|
|     F| 11| 0.3040404904876275|            0|   0|
|     F| 93| 0.6925469701105321|            0|   1|
|     F| 97|  0.839024350812226|            0|   1|
|     M| 12| 0.9340784848483313|            1|   1|
|     F| 81| 0.7937690320812052|            0|   1|
|     M| 35| 0.5292736282638121|            0|   1|
|     M| 12|0.03605670898047186|            0|   0|
|     M| 82|  0.548874333718212|            0|   1|
|     M| 71|

In [4]:
data.write.mode('overwrite').parquet('./test.parquet')

23/03/01 14:04:03 WARN MemoryManager: Total allocation exceeds 95.00% (964,768,551 bytes) of heap memory
Scaling row group sizes to 89.85% for 8 writers
23/03/01 14:04:03 WARN MemoryManager: Total allocation exceeds 95.00% (964,768,551 bytes) of heap memory
Scaling row group sizes to 79.87% for 9 writers
23/03/01 14:04:03 WARN MemoryManager: Total allocation exceeds 95.00% (964,768,551 bytes) of heap memory
Scaling row group sizes to 71.88% for 10 writers
23/03/01 14:04:03 WARN MemoryManager: Total allocation exceeds 95.00% (964,768,551 bytes) of heap memory
Scaling row group sizes to 65.35% for 11 writers
23/03/01 14:04:03 WARN MemoryManager: Total allocation exceeds 95.00% (964,768,551 bytes) of heap memory
Scaling row group sizes to 59.90% for 12 writers
23/03/01 14:04:03 WARN MemoryManager: Total allocation exceeds 95.00% (964,768,551 bytes) of heap memory
Scaling row group sizes to 65.35% for 11 writers
23/03/01 14:04:03 WARN MemoryManager: Total allocation exceeds 95.00% (964,768

In [5]:
from pandas import read_parquet

read_parquet('./test.parquet/')

,Gender,Age,Score,Binary_Target,Pred
0,F,13,0.802064,0,1
1,F,98,0.174615,1,0
2,M,20,0.734570,1,1
3,M,29,0.966650,0,1
4,F,37,0.979803,0,1
...,...,...,...,...,...
995,F,95,0.373565,0,0
996,F,75,0.858257,0,1
997,F,12,0.355249,0,0
998,F,95,0.158455,1,0


In [176]:
import pyspark
from tqdm import tqdm
from collections import defaultdict
from plotly.subplots import make_subplots
from plotly import graph_objects as go
from scipy import stats
from sklearn.metrics import confusion_matrix

class BinaryFairnessEval:
    '''Class for evaluating model fairness on categorical variables
    
    Constructor args:
        pred: name of column contraining hard predictions
        pred_proba: name of column containing probability scores 
        actual: name of column containing actual targets
        data: pyspark dataframe

    Methods:
     make_prevalence(sensitive_variable)
        Plots prevalence for categories using bootstrap samples

     make_equal_opportunity(sensitive_variable)
        Plots classification metrics for all categories

     make_independence(sensitive_variable)
        Prints Chi2 contingency test between sensitive variable and target

     make_separation(sensitive_variable)
        Plots separation of scores for sensitive variable


    Simple usage:

    b = BinaryFairnessEval(
        pred='Predicted', 
        pred_proba='Scores', 
        actual='Target',
        data=data
    )
    b.make_prevalence(sensitive_variable='Gender')
    
    '''
    def __init__(self,
        pred: str,
        pred_proba: str,
        actual: str,
        data: pyspark.sql.DataFrame) -> None:
        self.pred = pred # this is y_pred NOT y_pred_proba
        self.pred_proba = pred_proba # this is y_pred_proba NOT y_pred_proba_proba
        self.actual = actual # this is y_pred NOT y_pred_proba
        self.data = data


    def make_prevalence(self, sensitive_variable, n_samples=1000, fraction=0.1):
        pos_cnts = defaultdict(list) 
        neg_cnts = defaultdict(list) 

        for i in tqdm(range(n_samples)):
            sample = data.sample(fraction=fraction, withReplacement=True)

            grouped = sample.groupBy([self.pred, sensitive_variable]).count().toPandas()
            pos = grouped.query(f'{self.pred} == 1')
            neg = grouped.query(f'{self.pred} == 0')

            for cat in grouped[sensitive_variable].unique():
                pos_cnts[cat].extend(pos.loc[pos[sensitive_variable]==cat, 'count'])
                neg_cnts[cat].extend(neg.loc[neg[sensitive_variable]==cat, 'count'])

        cnts = {
            1: pos_cnts,
            0: neg_cnts
        }
        summaries = {}
        for target in [0, 1]: # HACK
            max_cnts = {} 
            min_cnts = {} 
            avg_cnts = {} 
            for cat in grouped[sensitive_variable].unique():
                max_cnts[cat] = np.max(cnts[target][cat])
                min_cnts[cat] = np.min(cnts[target][cat])
                avg_cnts[cat] = np.mean(cnts[target][cat])

            summaries[target] = {
                'max': max_cnts,
                'min': min_cnts,
                'avg': avg_cnts,
        }
        self._plot_prevalence(sensitive_variable, grouped, summaries)

    @staticmethod
    def fairness_metrics(data, pred, actual):
        """Calculate fairness for subgroup of population"""
        # from https://towardsdatascience.com/analysing-fairness-in-machine-learning-with-python-96a9ab0d0705
        
        #Confusion Matrix
        if isinstance(data, pyspark.sql.DataFrame):
            data = data.select(*[pred, actual]).toPandas()
        cm=confusion_matrix(df[actual],df[pred])
        TN, FP, FN, TP = cm.ravel()
        
        N = TP+FP+FN+TN #Total population
        ACC = (TP+TN)/N #Accuracy
        TPR = TP/(TP+FN) # True positive rate
        FPR = FP/(FP+TN) # False positive rate
        FNR = FN/(TP+FN) # False negative rate
        PPP = (TP + FP)/N # % predicted as positive
        
        return np.array([ACC, TPR, FPR, FNR, PPP])

    def make_equal_opportunity(self, sensitive_variable):
        df = self.data.toPandas()
        fig = go.Figure()
        for cat in df[sensitive_variable].unique():
            df_cat = df.loc[df[sensitive_variable]==cat, ]
            metrics_cat = BinaryFairnessEval.fairness_metrics(df, self.pred, self.actual)
            fig.add_trace(go.Bar(
                x=['ACC', 'TPR', 'FPR', 'FNR', 'PPP'],
                y=list(metrics_cat),
                name=cat
            ))

        fig.update_layout(
            barmode='group',
            title=f'Equal Opportunity for {sensitive_variable}'
        )
        fig.show()

    def _plot_prevalence(self, sensitive_variable, grouped, summaries):
        fig = go.Figure()
        fig.add_trace(go.Bar(
            x=grouped[sensitive_variable].unique(),
            y=[summaries[0]['avg'][c] for c in grouped[sensitive_variable].unique()], 
            name='Negative',
            error_y={
                'type':'data',
                'symmetric': False, 
                'array': [summaries[0]['max'][c] - summaries[0]['avg'][c] for c in grouped[sensitive_variable].unique()], 
                'arrayminus': [summaries[0]['min'][c] for c in grouped[sensitive_variable].unique()], 

            }
        ))

        fig.add_trace(go.Bar(
            x=grouped[sensitive_variable].unique(),
            y=[summaries[1]['avg'][c] for c in grouped[sensitive_variable].unique()], 
            name='Positive',
            error_y={
                'type':'data',
                'symmetric': False, 
                'array': [summaries[1]['max'][c] - summaries[1]['avg'][c] for c in grouped[sensitive_variable].unique()], 
                'arrayminus': [summaries[1]['min'][c] for c in grouped[sensitive_variable].unique()], 

            }
        ))

        fig.show()

    def make_independence(self, sensitive_variable):
        df = self.data.select(*[self.pred,self.pred_proba, self.actual, sensitive_variable]).toPandas()
        crosstab = pd.crosstab(df[self.pred], df[sensitive_variable])


        chi2, p, dof, expected = stats.chi2_contingency(crosstab)
        print(f'Chi2 Contingency Test for {sensitive_variable}')
        print(f'Chi2    :    {chi2}')
        print(f'p_value :    {p}')
        print(f'dof     :    {dof}')
        if p < 0.05:
            print(f'Little evidence to support dependence')
        if p >= 0.05 and p < 0.20:
            print(f'Some evidence to support dependence')
        else:
            print('Not enough evidence to disprove dependence')

    def make_separation(self, sensitive_variable):
        df = self.data.select(*[self.pred,self.pred_proba, self.actual, sensitive_variable]).toPandas()
        fig = make_subplots(rows=2, cols=1, subplot_titles=['Negative', 'Positive'])
        for cat in df[sensitive_variable].unique():
            fig.add_trace(
                go.Histogram(x=df.loc[
                    (df[sensitive_variable]==cat) & (df[self.actual]==0), 
                    self.pred_proba],
                    name=cat,
                ),
                row=1,
                col=1
            )
            fig.add_trace(
                go.Histogram(x=df.loc[
                    (df[sensitive_variable]==cat) & (df[self.actual]==1), 
                    self.pred_proba],
                    name=cat     
                ), 
                row=2,
                col=1
            )

        fig.update_layout(barmode='overlay')
        fig.update_traces(opacity=0.75)
        fig.show()



In [182]:
f = FairnessEval(actual='Binary_Target', pred='Pred', pred_proba='Score', data=data)
f.make_prevalence(sensitive_variable='Gender', n_samples=100)

Exception ignored in: <function tqdm.__del__ at 0x7f7f1d936d30>
Traceback (most recent call last):
  File "/home/aadi/miniconda3/envs/spark_env/lib/python3.8/site-packages/tqdm/std.py", line 1162, in __del__
    self.close()
  File "/home/aadi/miniconda3/envs/spark_env/lib/python3.8/site-packages/tqdm/notebook.py", line 288, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'


In [172]:
f.make_equal_opportunity(sensitive_variable='Gender')

In [173]:
f.make_independence(sensitive_variable='Gender')

Chi2 Contingency Test for Gender
Chi2    :    1.6527292210780082
p_value :    0.19858787622022533
dof     :    1
Some evidence to support dependence


In [174]:
f.make_separation(sensitive_variable='Gender')

In [49]:
summaries

{0: {'max': {'F': 40, 'M': 35},
  'min': {'F': 14, 'M': 11},
  'avg': {'F': 25.81, 'M': 23.56}},
 1: {'max': {'F': 38, 'M': 39},
  'min': {'F': 14, 'M': 10},
  'avg': {'F': 25.24, 'M': 27.44}}}

In [73]:
from plotly import graph_objects as go

fig = go.Figure()
fig.add_trace(go.Bar(
    x=grouped[sensitive_variable].unique(),
    y=[summaries[0]['avg'][c] for c in grouped[sensitive_variable].unique()], 
    name='Negative',
    error_y={
        'type':'data',
        'symmetric': False, 
        'array': [summaries[0]['max'][c] - summaries[0]['avg'][c] for c in grouped[sensitive_variable].unique()], 
        'arrayminus': [summaries[0]['min'][c] for c in grouped[sensitive_variable].unique()], 

    }
))

fig.add_trace(go.Bar(
    x=grouped[sensitive_variable].unique(),
    y=[summaries[1]['avg'][c] for c in grouped[sensitive_variable].unique()], 
    name='Positive',
    error_y={
        'type':'data',
        'symmetric': False, 
        'array': [summaries[1]['max'][c] - summaries[1]['avg'][c] for c in grouped[sensitive_variable].unique()], 
        'arrayminus': [summaries[1]['min'][c] for c in grouped[sensitive_variable].unique()], 

    }
))

